**Задача**

Предварительная обработка не требуется, данные уже закодированы и нормализованы.

В качестве метрики будем использовать F1-score.

Необходимо обучить две модели: логистическую регрессию и случайный лес. Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. Важно использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций не должно превышать 50.

In [1]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

%matplotlib inline
plt.style.use('seaborn')

In [2]:
data = pd.read_csv('_train_sem09.csv', sep=',')
data.head(3)

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = data.drop(['Activity'], axis=1)
y = data['Activity']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42, test_size=0.2)

In [46]:
#Создаем объект класса логистическая регрессия
# по условию задачи максимальное количество итераций не должно превышать 50
log_reg = linear_model.LogisticRegression(max_iter = 50)
#Обучаем модель, минимизируя logloss
log_reg.fit(X_train, y_train)
print("accuracy на тренировочном наборе: {:.2f}".format(log_reg.score(X_train, y_train)))
y_train_pred = log_reg.predict(X_train)
y_test_pred = log_reg.predict(X_test)
print("accuracy на тестовом наборе: {:.2f}".format(metrics.accuracy_score(y_test, y_test_pred)))
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
#accuracy на тренировочном наборе: 0.88
#accuracy на тестовом наборе: 0.75
#f1_score на тестовом наборе: 0.78

accuracy на тренировочном наборе: 0.86
accuracy на тестовом наборе: 0.75
f1_score на тренировочном наборе: 0.88
f1_score на тестовом наборе: 0.78


c:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [47]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=42)

#Обучаем модель
rf.fit(X_train, y_train)
#Выводим значения метрики
print("accuracy на тренировочном наборе: {:.2f}".format(log_reg.score(X_train, y_train)))
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)
print("accuracy на тестовом наборе: {:.2f}".format(metrics.accuracy_score(y_test, y_test_pred)))
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

accuracy на тренировочном наборе: 0.86
accuracy на тестовом наборе: 0.79
f1_score на тренировочном наборе: 1.00
f1_score на тестовом наборе: 0.80


### <center> **GridSearchCV**

Настроим оптимизацию гиперпараметров для алгоритма логистической регрессии.

In [48]:
from sklearn.model_selection import GridSearchCV

param_grid = {'penalty': ['l2', 'none'] ,#тип регурялизации
              'solver': ['lbfgs', 'saga'], #алгоритм оптимизации
              }
grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=42, #генератор случайных чисел
        # по условию задачи максимальное количество итераций не должно превышать 50
        max_iter=50 #количество итераций на сходимость
    ), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
) 
%time grid_search.fit(X_train_scaled, y_train) 
print("accuracy на тренировочном наборе: {:.2f}".format(grid_search.score(X_train, y_train)))
y_train_pred = grid_search.predict(X_train)
y_test_pred = grid_search.predict(X_test)
print("accuracy на тестовом наборе: {:.2f}".format(metrics.accuracy_score(y_test, y_test_pred)))
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

CPU times: total: 4.08 s
Wall time: 27 s
accuracy на тренировочном наборе: 0.61
accuracy на тестовом наборе: 0.61
f1_score на тренировочном наборе: 0.74
f1_score на тестовом наборе: 0.73
Наилучшие значения гиперпараметров: {'penalty': 'l2', 'solver': 'saga'}


c:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
c:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
c:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


Настроим оптимизацию гиперпараметров для алгоритма случайного леса.

In [49]:
param_grid = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': [5],
              'max_depth': list(np.linspace(20, 40, 5, dtype=int))
              }
            
grid_search_forest = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time 
grid_search_forest.fit(X_train, y_train) 
print("accuracy на тренировочном наборе: {:.2f}".format(grid_search_forest.score(X_train, y_train)))
y_train_pred = grid_search_forest.predict(X_train)
y_test_pred = grid_search_forest.predict(X_test)
print("accuracy на тестовом наборе: {:.2f}".format(metrics.accuracy_score(y_test, y_test_pred)))
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_forest.best_params_))

CPU times: total: 0 ns
Wall time: 0 ns
accuracy на тренировочном наборе: 0.94
accuracy на тестовом наборе: 0.78
f1_score на тренировочном наборе: 0.94
f1_score на тестовом наборе: 0.80
Наилучшие значения гиперпараметров: {'max_depth': 20, 'min_samples_leaf': 5, 'n_estimators': 140}


### <center> **RandomizedSearchCV**

Настроим оптимизацию гиперпараметров для алгоритма логистической регрессии.

In [52]:
from sklearn.model_selection import RandomizedSearchCV

#np.linspace(start(от), stop(до), num=50(количество),dtype- тип данных)
param_grid = {'penalty': ['l2', 'none'] ,#тип регурялизации
              'solver': ['lbfgs', 'saga'], #алгоритм оптимизации
              }
     
random_search = RandomizedSearchCV(
    # по условию задачи максимальное количество итераций не должно превышать 50
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=50), 
    param_distributions=param_grid, 
    cv=5, 
    n_iter = 10, 
    n_jobs = -1
)  
%time 
random_search.fit(X_train, y_train) 
print("accuracy на тренировочном наборе: {:.2f}".format(random_search.score(X_train, y_train)))
y_train_pred = random_search.predict(X_train)
y_test_pred = random_search.predict(X_test)
print("accuracy на тестовом наборе: {:.2f}".format(metrics.accuracy_score(y_test, y_test_pred)))
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

CPU times: total: 0 ns
Wall time: 0 ns


c:\Users\USER\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


accuracy на тренировочном наборе: 0.85
accuracy на тестовом наборе: 0.76
f1_score на тренировочном наборе: 0.87
f1_score на тестовом наборе: 0.78
Наилучшие значения гиперпараметров: {'solver': 'saga', 'penalty': 'l2'}


c:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Настроим оптимизацию гиперпараметров для алгоритма случайного леса.

In [53]:
param_grid = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': [5],
              'max_depth': list(np.linspace(20, 40, 10, dtype=int))
              }
            
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_grid, 
    cv=5,
    n_iter = 10, 
    n_jobs = -1
)  
%time 
random_search_forest.fit(X_train, y_train) 
print("accuracy на тренировочном наборе: {:.2f}".format(random_search_forest.score(X_train, y_train)))
y_train_pred = random_search_forest.predict(X_train)
y_test_pred = random_search_forest.predict(X_test)
print("accuracy на тестовом наборе: {:.2f}".format(metrics.accuracy_score(y_test, y_test_pred)))
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))

CPU times: total: 0 ns
Wall time: 0 ns
accuracy на тренировочном наборе: 0.94
accuracy на тестовом наборе: 0.78
f1_score на тренировочном наборе: 0.94
f1_score на тестовом наборе: 0.80
Наилучшие значения гиперпараметров: {'n_estimators': 140, 'min_samples_leaf': 5, 'max_depth': 20}


### <center> Hyperopt

In [5]:
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials

Настроим оптимизацию гиперпараметров для алгоритма логистической регрессии.

In [6]:
# зададим пространство поиска гиперпараметров
space={'penalty': hp.choice('penalty', ['l2', 'none']),
       'solver' : hp.choice('solver', ['lbfgs', 'saga']),
      }

In [7]:
# зафиксируем random_state
random_state = 42
def hyperopt_log(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'penalty': params['penalty'], 
              'solver': params['solver']
              }
    # используем эту комбинацию для построения модели
    model = linear_model.LogisticRegression(**params, random_state=random_state)
    # обучаем модель
    #model.fit(X, y)
    #score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
    
    return -score

In [10]:
# начинаем подбор гиперпараметров
%time
trials = Trials() # используется для логирования результатов
best=fmin(hyperopt_log, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.RandomState(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

CPU times: total: 0 ns
Wall time: 0 ns
100%|██████████| 20/20 [02:53<00:00,  8.68s/trial, best loss: -0.7793179449996321]
Наилучшие значения гиперпараметров {'penalty': 0, 'solver': 1}


In [18]:
# рассчитаем точность для тестовой выборки
model_log = linear_model.LogisticRegression(
    random_state=random_state, 
    penalty='l2' if best['penalty']==0 else 'none',
    solver='lbfgs' if best['solver']==0 else 'saga',
    max_iter=50 # по условию задачи максимальное количество итераций не должно превышать 50
)
model_log.fit(X_train, y_train)
print("accuracy на тренировочном наборе: {:.2f}".format(model_log.score(X_train, y_train)))
y_train_pred = model_log.predict(X_train)
y_test_pred = model_log.predict(X_test)
print("accuracy на тестовом наборе: {:.2f}".format(metrics.accuracy_score(y_test, y_test_pred)))
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

accuracy на тренировочном наборе: 0.85
accuracy на тестовом наборе: 0.76
f1_score на тренировочном наборе: 0.87
f1_score на тестовом наборе: 0.78


c:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Настроим оптимизацию гиперпараметров для алгоритма случайного леса.

In [122]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

In [123]:
# зафиксируем random_state
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)
    # обучаем модель
    #model.fit(X, y)
    #score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
    
    return -score

In [125]:
# начинаем подбор гиперпараметров
%time
trials = Trials() # используется для логирования результатов
best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.RandomState(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

CPU times: total: 0 ns
Wall time: 0 ns
100%|██████████| 20/20 [02:02<00:00,  6.15s/trial, best loss: -0.8157677360251723]
Наилучшие значения гиперпараметров {'max_depth': 24.0, 'min_samples_leaf': 2.0, 'n_estimators': 153.0}


In [126]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train)
print("accuracy на тренировочном наборе: {:.2f}".format(model.score(X_train, y_train)))
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
print("accuracy на тестовом наборе: {:.2f}".format(metrics.accuracy_score(y_test, y_test_pred)))
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

accuracy на тренировочном наборе: 0.99
accuracy на тестовом наборе: 0.79
f1_score на тренировочном наборе: 0.99
f1_score на тестовом наборе: 0.81


## <center> Optuna

In [19]:
import optuna

Настроим оптимизацию гиперпараметров для алгоритма логистической регрессии.

In [55]:
def optuna_log(trial):
  # задаем пространства поиска гиперпараметров
  penalty = trial.suggest_categorical('penalty', ['l2', 'none'])
  solver = trial.suggest_categorical('solver', ['lbfgs', 'saga'])

  # создаем модель
  model = linear_model.LogisticRegression(penalty=penalty,
                                          solver=solver,
                                          max_iter=50) # по условию задачи максимальное количество итераций не должно превышать 50
  # обучаем модель
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))

  return score

In [56]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_log, n_trials=20)

[I 2022-06-14 21:19:08,273] A new study created in memory with name: LogisticRegression
c:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2022-06-14 21:19:12,472] Trial 0 finished with value: 0.866303690260133 and parameters: {'penalty': 'l2', 'solver': 'saga'}. Best is trial 0 with value: 0.866303690260133.
c:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2022-06-14 21:19:16,804] Trial 1 finished with value: 0.8659606656580938 and parameters: {'penalty': 'l2', 'solver': 'saga'}. Best is trial 0 with value: 0.866303690260133.
c:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of 

CPU times: total: 55.5 s
Wall time: 37.2 s


In [57]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

Наилучшие значения гиперпараметров {'penalty': 'none', 'solver': 'lbfgs'}
f1_score на обучающем наборе: 0.88


In [67]:
# рассчитаем точность для тестовой выборки
# по условию задачи максимальное количество итераций не должно превышать 50
model_opt_log = linear_model.LogisticRegression(**study.best_params, random_state=random_state, max_iter=50)
model_opt_log.fit(X_train, y_train)
y_train_pred = model_opt_log.predict(X_train)
print("accuracy на тренировочном наборе: {:.2f}".format(model_opt_log.score(X_train, y_train)))
y_train_pred = model_opt_log.predict(X_train)
y_test_pred = model_opt_log.predict(X_test)
print("accuracy на тестовом наборе: {:.2f}".format(metrics.accuracy_score(y_test, y_test_pred)))
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

c:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


accuracy на тренировочном наборе: 0.87
accuracy на тестовом наборе: 0.74
f1_score на тренировочном наборе: 0.88
f1_score на тестовом наборе: 0.77


Настроим оптимизацию гиперпараметров для алгоритма случайного леса.

In [20]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 10, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))

  return score

In [21]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

[I 2022-06-14 17:08:25,154] A new study created in memory with name: RandomForestClassifier
[I 2022-06-14 17:08:27,776] Trial 0 finished with value: 0.9503805175038051 and parameters: {'n_estimators': 114, 'max_depth': 16, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.9503805175038051.
[I 2022-06-14 17:08:30,984] Trial 1 finished with value: 0.9256550883607556 and parameters: {'n_estimators': 156, 'max_depth': 15, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.9503805175038051.
[I 2022-06-14 17:08:33,807] Trial 2 finished with value: 0.9025796661608498 and parameters: {'n_estimators': 144, 'max_depth': 30, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.9503805175038051.
[I 2022-06-14 17:08:37,612] Trial 3 finished with value: 0.9009118541033436 and parameters: {'n_estimators': 191, 'max_depth': 16, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.9503805175038051.
[I 2022-06-14 17:08:41,632] Trial 4 finished with value: 0.9749847467968273 and parameters: {'n_

CPU times: total: 1min 12s
Wall time: 1min 16s


In [22]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

Наилучшие значения гиперпараметров {'n_estimators': 165, 'max_depth': 24, 'min_samples_leaf': 2}
f1_score на обучающем наборе: 0.99


In [23]:
# рассчитаем точность для тестовой выборки
model_opt_rf = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model_opt_rf.fit(X_train, y_train)
y_train_pred = model_opt_rf.predict(X_train)
print("accuracy на тренировочном наборе: {:.2f}".format(model_opt_rf.score(X_train, y_train)))
y_train_pred = model_opt_rf.predict(X_train)
y_test_pred = model_opt_rf.predict(X_test)
print("accuracy на тестовом наборе: {:.2f}".format(metrics.accuracy_score(y_test, y_test_pred)))
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

accuracy на тренировочном наборе: 0.99
accuracy на тестовом наборе: 0.79
f1_score на тренировочном наборе: 0.99
f1_score на тестовом наборе: 0.81
